### Introduction
This notebook contains a simple GUI constructing using Jupyter Widgets intended to help with testing, demonstrate the functionality to non-technical users, and act as an example of using the Python API to technical users. It is not intended as a production usage GUI and the Python code behind the GUI is not of production quality and robustness.

Each input tab of the GUI contains buttons to save/load inputs to/from CSV files in order to avoid repeatedly typing in the same data each time the GUI is launched. Some example of such persisted data can be found in the directory ./storage_gui_data in the same directory as this notebook.

After installing the prerequisites listed below, simply run the single cell and the GUI will appear.

#### Prerequisites
This notebook requires several packages installed:

`
pip install cmdty-storage
pip install curves
pip install ipysheet
`

If this notebook has been downloaded standalone, i.e. without downloading the whole storage repo,
it requires the file [storage_gui.py](https://github.com/cmdty/storage/blob/master/samples/python/storage_gui.py) be in the same directory.


In [1]:
%matplotlib inline
import storage_gui as sg
sg.display_gui()

In [33]:
import pandas as pd
from cmdty_storage import CmdtyStorage, intrinsic_value

monthly_index = pd.period_range(start='2023-04-01', periods=25, freq='M')
monthly_fwd_prices = [16.61, 15.68, 15.42, 15.31, 15.27, 15.13, 15.96, 17.22, 17.32, 17.66, 
                      17.59, 16.81, 15.36, 14.49, 14.28, 14.25, 14.32, 14.33, 15.30, 16.58, 
                      16.64, 16.79, 16.64, 15.90, 14.63]
fwd_curve = pd.Series(data=monthly_fwd_prices, index=monthly_index).resample('D').fillna('pad')

rates = [0.005, 0.006, 0.0072, 0.0087, 0.0101, 0.0115, 0.0126]
rates_pillars = pd.PeriodIndex(freq='D', data=['2021-04-25', '2021-06-01', '2021-08-01', '2021-12-01', '2022-04-01', 
                                             '2022-12-01', '2025-12-01'])

ir_curve = pd.Series(data=rates, index=rates_pillars).resample('D').asfreq('D').interpolate(method='linear')

def settlement_rule(delivery_date):
    return delivery_date.asfreq('M').asfreq('D', 'end') + 20

max_inventory = 1.0 # MWh

injection_duration = 100.0 # days
withdrawal_duration = injection_duration / 2.0 # days

max_injection_rate = max_inventory / injection_duration # MWh / day
max_withdrawal_rate = max_inventory / withdrawal_duration # MWh / day

yearly_period_index = pd.PeriodIndex(freq='D', data = ['2023-04-01', '2023-10-01', '2024-04-01'])

min_inventory = pd.Series([0.0, 0.03 * max_inventory, 0.0], yearly_period_index).resample('D').fillna('pad')
#max_inventory = pd.Series([max_inventory, max_inventory, max_inventory], yearly_period_index).resample('D').fillna('pad')

print(min_inventory)
print(max_withdrawal_rate)

storage_time_varying = CmdtyStorage(
freq='D',
storage_start = '2023-04-01',
storage_end = '2024-04-01',
injection_cost = 0.0,
withdrawal_cost = 0.0,
min_inventory = pd.Series([0.0, 0.03 * max_inventory, 0.0], yearly_period_index).resample('D').fillna('pad'),
max_inventory = pd.Series([max_inventory, max_inventory, max_inventory], yearly_period_index).resample('D').fillna('pad'),
max_injection_rate = max_injection_rate,
max_withdrawal_rate = max_withdrawal_rate,
terminal_storage_npv = lambda spot_price, inventory: 0.97 * spot_price * inventory
)

iv_results = intrinsic_value(
cmdty_storage = storage_time_varying,
val_date = storage_time_varying.start,
inventory = 0.0 * max_inventory,
forward_curve = fwd_curve,
interest_rates = ir_curve,
settlement_rule = settlement_rule)

iv_results

#pd.Series([0.0, 0.03 * max_inventory, 0.0], yearly_period_index).resample('D').fillna('pad'),

2023-04-01    0.00
2023-04-02    0.00
2023-04-03    0.00
2023-04-04    0.00
2023-04-05    0.00
              ... 
2024-03-28    0.03
2024-03-29    0.03
2024-03-30    0.03
2024-03-31    0.03
2024-04-01    0.00
Freq: D, Length: 367, dtype: float64
0.02


IntrinsicValuationResults(npv=2.2290107338585408, profile=            inventory  inject_withdraw_volume  cmdty_consumed  inventory_loss  \
2023-04-01       0.01                    0.01             0.0             0.0   
2023-04-02       0.02                    0.01             0.0             0.0   
2023-04-03       0.03                    0.01             0.0             0.0   
2023-04-04       0.04                    0.01             0.0             0.0   
2023-04-05       0.05                    0.01             0.0             0.0   
...               ...                     ...             ...             ...   
2024-03-28       0.04                   -0.02             0.0             0.0   
2024-03-29       0.03                   -0.01             0.0             0.0   
2024-03-30       0.03                    0.00             0.0             0.0   
2024-03-31       0.01                   -0.02             0.0             0.0   
2024-04-01       0.01                    0.00      